# BIGRoC: Boosting Image Generation via a Robust Classifier

--------------------------------------------------

## Boosting deep BigGAN on ImageNet $256\times256$

This colab notebook contains the needed code to experiment with our proposed algorithm for boosting deep BigGAN on ImageNet 256x256(in Section 5.2).

**How to use**

1.   Upload the notebook to colab
2.   Make sure that colab uses a GPU (Edit $\rightarrow$ Notebook settings $\rightarrow$ Hardware accelerator)

**Setup:**

This notebook mounts google drive and assumes that [this directory](https://drive.google.com/drive/folders/1yN6WjMmc-pi3zHylF-I1jri7I2nZpsGJ?usp=sharing) is located in your root folder in Google Drive. Therefore, please open the Google Drive's link and choose "add a shortcut to drive" and pick "My Drive" is the chosen location. By doing so, you are ready to go :)

If you rather not to mount your google drive, download the files and edit the relevant paths accordingly.

**Credits**

This notebook uses the following packages:

* [robustness](https://github.com/MadryLab/robustness) - For adversarially robust classifier
* [guided-diffuision](https://github.com/openai/guided-diffusion/tree/main/evaluations) - ImageNet quantitative evaluation and 50K set of generated images + labels

# Installation & Setup

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install torch==1.8.2+cu102 torchvision==0.9.2+cu102 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

In [ ]:
!wget http://andrewilyas.com/ImageNet.pt

In [ ]:
!pip install robustness

In [ ]:
from robustness import model_utils, datasets
import torch


def create_dl_model(DATA='CIFAR', BATCH_SIZE=128, NUM_WORKERS=8):
    '''
    :param DATA: Choices: ['CIFAR', 'ImageNet', 'RestrictedImageNet']
    :param bs: batch size
    :param num_workers:
    :return: a dataloader object
    '''

    # Load dataset
    dataset_function = getattr(datasets, DATA)
    dataset = dataset_function('data')
    # Load model
    model_kwargs = {
        'arch': 'resnet50',
        'dataset': dataset,
        'resume_path': f'./{DATA}.pt'
    }
    model, _ = model_utils.make_and_restore_model(**model_kwargs)
    classifier = model.model
    classifier.eval()
    return classifier

adv_model = create_dl_model(DATA='ImageNet')

In [6]:
import torch
from torch import nn


class AttackerStep:
    '''
    Generic class for attacker steps, under perturbation constraints
    specified by an "origin input" and a perturbation magnitude.
    Must implement project, step, and random_perturb
    '''

    def __init__(self, orig_input, eps, step_size, use_grad=True):
        '''
        Initialize the attacker step with a given perturbation magnitude.
        Args:
            eps (float): the perturbation magnitude
            orig_input (ch.tensor): the original input
        '''
        self.orig_input = orig_input
        self.eps = eps
        self.step_size = step_size
        self.use_grad = use_grad

    def project(self, x):
        '''
        Given an input x, project it back into the feasible set
        Args:
            ch.tensor x : the input to project back into the feasible set.
        Returns:
            A `ch.tensor` that is the input projected back into
            the feasible set, that is,
        .. math:: \min_{x' \in S} \|x' - x\|_2
        '''
        raise NotImplementedError

    def step(self, x, g):
        '''
        Given a gradient, make the appropriate step according to the
        perturbation constraint (e.g. dual norm maximization for :math:`\ell_p`
        norms).
        Parameters:
            g (ch.tensor): the raw gradient
        Returns:
            The new input, a ch.tensor for the next step.
        '''
        raise NotImplementedError

    def random_perturb(self, x):
        '''
        Given a starting input, take a random step within the feasible set
        '''
        raise NotImplementedError

    def to_image(self, x):
        '''
        Given an input (which may be in an alternative parameterization),
        convert it to a valid image (this is implemented as the identity
        function by default as most of the time we use the pixel
        parameterization, but for alternative parameterizations this functino
        must be overriden).
        '''
        return x


from torch import nn

# simple Module to normalize an image
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean)
        self.std = torch.Tensor(std)
    def forward(self, x):
        return (x - self.mean.type_as(x)[None, :, None, None]) / self.std.type_as(x)[None, :, None, None]

norm = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])



# L2 threat model
class L2Step(AttackerStep):
    """
    Attack step for :math:`\ell_\infty` threat model. Given :math:`x_0`
    and :math:`\epsilon`, the constraint set is given by:
    .. math:: S = \{x | \|x - x_0\|_2 \leq \epsilon\}
    """

    def project(self, x):
        """
        """
        if self.orig_input is None: self.orig_input = x.detach()
        self.orig_input = self.orig_input.cuda()
        diff = x - self.orig_input
        diff = diff.renorm(p=2, dim=0, maxnorm=self.eps)
        return torch.clamp(self.orig_input + diff, 0, 1)

    def step(self, x, g):
        """
        """
        l = len(x.shape) - 1
        g_norm = torch.norm(g.reshape(g.shape[0], -1), dim=1).view(-1, *([1] * l))
        scaled_g = g / (g_norm + 1e-10)
        return x + scaled_g * self.step_size

def targeted_pgd_l2(model, X, y, num_iter, eps, step_size):
    # input images are in range [0,1]
    steper = L2Step(eps=eps, orig_input=None, step_size=step_size)
    for t in range(num_iter):
        X = X.clone().detach().requires_grad_(True).cuda()
        loss = nn.CrossEntropyLoss(reduction='none')(model(norm(X)), y)
        loss = torch.mean(loss)
        grad, = torch.autograd.grad(-1 * loss, [X])
        X = steper.step(X, grad)
        X = steper.project(X)
    return X.detach()

In [ ]:
!wget https://openaipublic.blob.core.windows.net/diffusion/jul-2021/ref_batches/imagenet/256/biggan_deep_trunc1_imagenet256.npz

In [8]:
import numpy as np

x = np.load("biggan_deep_trunc1_imagenet256.npz", mmap_mode='r') 
arr_orig = x['arr_0']
gt_labels = x['arr_1']

In [ ]:
arr_orig.shape, arr_orig.min(), arr_orig.max()

Download Reference Batch

In [ ]:
!wget https://openaipublic.blob.core.windows.net/diffusion/jul-2021/ref_batches/imagenet/256/VIRTUAL_imagenet256_labeled.npz

# Deep BigGAN baseline quantitative results

Deep BigGAN eval

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./biggan_deep_trunc1_imagenet256.npz

downloading InceptionV3 model...
11679it [00:12, 942.92it/s]
2022-01-15 07:50:33.069156: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:08<00:00,  8.63s/it]
computing reference batch activations...
100% 157/157 [00:32<00:00,  4.78it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:44<00:00,  4.74it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 202.647216796875
FID: 7.034064226776422
sFID: 7.291727678621896
tcmalloc: large alloc 2000003072 bytes == 0x55be19e2a000 @  0x7ffaac1ba001 0x7ffaa8c8254f 0x7ffaa8cd2b58 0x7ffaa8cd6b17 0x7ffaa8d75203 0x55bd44d62544 0x55bd44d62240 0x55bd44dd6627 0x55bd44dd0ced 0x55bd44d63bda 0x55bd44dd1c0d 0x55bd44d63afa 0x55bd44dd1c0d 0x55bd44d63afa 0x55bd44dd1915 0x55bd44dd09ee 0x55bd44dd06f3 0x55bd

# BIGRoC Application

In [1]:
# BIGRoC args
epsilon, steps = 1, 7
step_size = (epsilon * 1.5) / steps

In [11]:
from tqdm.notebook import tqdm

torch.manual_seed(1234)
for k in range(50):
  boosted_gen_imgs = []
  for i in tqdm(range(100)):
    x_batch = arr_orig[(1000*k) + i * 10: (1000*k) + (i + 1) * 10]
    x_batch_0_1 = torch.tensor(x_batch / 255.).cuda().permute(0,3,1,2).float()
    with torch.no_grad():
      # labels = torch.argmax(adv_model(norm(x_batch_0_1)), dim=1)
      labels = torch.tensor(gt_labels[(1000*k) + i * 10: (1000*k) + (i + 1) * 10]).cuda().long()
    b_imgs = targeted_pgd_l2(model=adv_model, X=x_batch_0_1.data, y=labels.long(), num_iter=steps, eps=epsilon,
                                      step_size=step_size).detach().cpu()
    boosted_gen_imgs.append(b_imgs)

  boosted_gen_imgs = torch.cat(boosted_gen_imgs)
  boosted_gen_imgs = (boosted_gen_imgs * 255.).int()
  boosted_gen_imgs = boosted_gen_imgs.detach().cpu().permute(0,2,3,1).numpy().astype(np.uint8)
  np.savez(f"./boosted_biggan_deep_256_{k}_eps_1_steps_{steps}_labeled", boosted_gen_imgs)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

load boosted images

In [3]:
import numpy as np
from tqdm.notebook import tqdm

arr = np.zeros(shape=(50000, 256, 256, 3), dtype=np.uint8)
for f in tqdm(range(50)):
  x = np.load(f"./boosted_biggan_deep_256_{f}_eps_1_steps_{steps}_labeled.npz")['arr_0']
  arr[1000 * f: 1000 * (f+1)] = x
  x = None

np.savez(f"boosted_biggan_deep_256_eps_1_steps_{steps}", arr)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
arr.shape, arr.min(), arr.max()

$\epsilon = 1$, BIGRoC$_{GT}$

In [6]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./boosted_biggan_deep_256_eps_1_steps_7.npz

2022-01-17 06:12:49.488032: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:02<00:00,  2.02s/it]
computing reference batch activations...
100% 157/157 [00:32<00:00,  4.83it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:45<00:00,  4.72it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 228.23031616210938
FID: 6.844337438340631
sFID: 7.294456415571744
tcmalloc: large alloc 2000003072 bytes == 0x55e96e0c6000 @  0x7f380eba4001 0x7f380b66c54f 0x7f380b6bcb58 0x7f380b6c0b17 0x7f380b75f203 0x55e899f38544 0x55e899f38240 0x55e899fac627 0x55e899fa6ced 0x55e899f39bda 0x55e899fa7c0d 0x55e899f39afa 0x55e899fa7c0d 0x55e899f39afa 0x55e899fa7915 0x55e899fa69ee 0x55e899fa66f3 0x55e89a0704c2 0x55e89a07083d 0x55e89a0706e6 0x55e89a048163 0x55e

$\epsilon = 1$, BIGRoC$_{PL}$

In [ ]:
!python ./gdrive/MyDrive/models_and_scripts/evaluator.py ./VIRTUAL_imagenet256_labeled.npz ./boosted_biggan_deep_256_eps_1_steps_7.npz

downloading InceptionV3 model...
11679it [00:16, 698.93it/s]
2022-01-16 08:55:19.945134: W tensorflow/core/framework/op_def_util.cc:371] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
warming up TensorFlow...
100% 1/1 [00:08<00:00,  8.77s/it]
computing reference batch activations...
100% 157/157 [00:34<00:00,  4.53it/s]
computing/reading reference batch statistics...
computing sample batch activations...
100% 782/782 [02:47<00:00,  4.67it/s]
computing/reading sample batch statistics...
Computing evaluations...
Inception Score: 221.7795867919922
FID: 6.928997517627522
sFID: 7.297275309189331
tcmalloc: large alloc 2000003072 bytes == 0x557eacd5c000 @  0x7f6b6ba4d001 0x7f6b6851554f 0x7f6b68565b58 0x7f6b68569b17 0x7f6b68608203 0x557e09695544 0x557e09695240 0x557e09709627 0x557e09703ced 0x557e09696bda 0x557e09704c0d 0x557e09696afa 0x557e09704c0d 0x557e09696afa 0x557e09704915 0x557e097039ee 0x557e097036f3 0x557